## 실험 환경 정보

In [14]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

print("===== Environment Info =====")
print("Pandas version:", pd.__version__)
print("Numpy version:", np.__version__)
print("Matplotlib version:", matplotlib.__version__)
print("==========================")

===== Environment Info =====
Pandas version: 2.2.3
Numpy version: 1.23.5
Matplotlib version: 3.10.3


## 실험 코드

In [2]:

# 1. CSV 불러오기 (세미콜론 구분자 사용)
ab_test_df = pd.read_csv('ab_test.csv', sep=';')
auth_data_df = pd.read_csv('auth_data.csv', sep=';')
reg_data_df = pd.read_csv('reg_data.csv', sep=';')

# 2. timestamp 컬럼을 datetime 형식으로 변환
auth_data_df['auth_ts'] = pd.to_datetime(auth_data_df['auth_ts'], unit='s')
reg_data_df['reg_ts'] = pd.to_datetime(reg_data_df['reg_ts'], unit='s')

# 3. 로그인 요약 정보 생성
login_summary = auth_data_df.groupby('uid').agg(
    login_count=('auth_ts', 'count'),
    first_login=('auth_ts', 'min'),
    last_login=('auth_ts', 'max')
).reset_index()

# 4. 컬럼명 정리
reg_data_df.rename(columns={'reg_ts': 'reg_date'}, inplace=True)

# 5. 데이터 병합
merged_df = ab_test_df.merge(reg_data_df, left_on='user_id', right_on='uid', how='left')
merged_df = merged_df.merge(login_summary, left_on='user_id', right_on='uid', how='left')
merged_df.drop(columns=['uid_x', 'uid_y'], inplace=True)

# 6. 컬럼 순서 정리
merged_df = merged_df[['user_id', 'testgroup', 'revenue', 'reg_date', 'login_count', 'first_login', 'last_login']]

# 7. 저장
merged_df.to_csv('merged_abtest_data.csv', index=False)


In [11]:
merged_df = pd.read_csv("merged_abtest_data.csv", parse_dates=["reg_date", "first_login", "last_login"])

# A/B 테스트 그룹별 유저 수
group_counts = merged_df["testgroup"].value_counts()
group_counts

testgroup
b    202667
a    202103
Name: count, dtype: int64

In [12]:
# A/B 그룹별 평균 매출
avg_revenue_by_group = merged_df.groupby("testgroup")["revenue"].mean()
avg_revenue_by_group

testgroup
a    25.413720
b    26.751287
Name: revenue, dtype: float64

In [13]:
# 매출 발생 유저 비율
nonzero_revenue_ratio = (merged_df["revenue"] > 0).mean()
nonzero_revenue_ratio

0.009222521431924303

In [10]:
# 월별 가입자 수
reg_month_dist = merged_df["reg_date"].dt.to_period("M").value_counts().sort_index()
reg_month_dist

reg_date
1998-11        1
1999-07        1
2000-01        1
2000-05        1
2000-09        1
           ...  
2018-09    14645
2018-10    15911
2018-11    16189
2018-12    17590
2019-01    10620
Freq: M, Name: count, Length: 215, dtype: int64

In [9]:
# 로그인 횟수별 평균 매출 (상위 10)
revenue_by_login_count = merged_df.groupby("login_count")["revenue"].mean().sort_index().head(10)
revenue_by_login_count.head(10)

login_count
1.0     27.060254
2.0     13.459859
3.0     26.772001
4.0     40.226326
5.0     32.069203
6.0     24.991214
7.0     19.799855
8.0     21.801879
9.0     30.823984
10.0    28.031599
Name: revenue, dtype: float64